<a href="https://colab.research.google.com/github/zanevc5555/physics-applets/blob/main/SpinEnsembles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
x=1
print(x)

1
